#### ML-DOCTOR: Holistic Risk Assessment of Inference Attacks Against Machine Learning Models

##### 1. Published on: USENIX 22 summer

##### 2. Contribution:

- 首个针对不同的机器学习模型inference attacks进行全面的风险分析模型。

    - 四种攻击方式:
        - [membership inference](https://arxiv.org/pdf/1610.05820)，记作$MemInf$
        
        - [model inversion](https://dl.acm.org/doi/pdf/10.1145/2810103.2813677)，记作$ModInv$

        - [attribute inference](https://arxiv.org/pdf/1805.04049)，记作$AttrInf$

        - [model stealing](https://www.usenix.org/system/files/conference/usenixsecurity16/sec16_paper_tramer.pdf)，记作$ModSteal$

    - 针对图像数据集进行测试: 为了保证公平性，所有图像数据都被设置为32x32x3的大小
        
        - CelebA

        - Fashion-MNIST

        - STL10

        - UTKFace

    - 模型结构：

        - [AlexNet](https://dl.acm.org/doi/abs/10.1145/3065386)

        - [ResNet18](http://openaccess.thecvf.com/content_cvpr_2016/html/He_Deep_Residual_Learning_CVPR_2016_paper.html)

        - [VGG19](https://arxiv.org/abs/1409.1556)

        - [Xception](http://openaccess.thecvf.com/content_cvpr_2017/html/Chollet_Xception_Deep_Learning_CVPR_2017_paper.html)
        
        - SimpleCNN（2个卷积，2个全连接）

- 创建了一个威胁级别分类系统(分类系统)

- 发布ML-DOCTOR使得模型持有方能够评估模型受到不同攻击后可能承担的风险

##### 3. Questions & Answers & Findings

Q&A:

- What is the impact of dataset complexity on different attacks? 

    - 训练集的复杂程度与membership inference, model inversion和model stealing攻击的效果有很大关系，特别是membership inference，该攻击在复杂数据集上表现的非常好，而其他攻击方式则在简单数据集上表现的更好。

- What is the impact of overfitting on different attacks? 

    - 由于复杂数据集过拟合的可能性相较于简单数据集小，因此membership infernce效果会更好一些。

- What is the relationship among different attacks?

    - DP-SGD和知识蒸馏都能够减轻membership inference，知识蒸馏效果会比DP-SGD差一些，但他们在其他的inference attacks上的表现并不是很好。这篇文章解释了他们能够保护的程度以及为什么他们能有这种保护。

Findings:

- membership inference和model stealing attack的表现是负相关的（相关系数为-0.821）

- 获得一部分数据集并不会给membership inference, attribute inference和model stealing带来特别多的提升


##### 4. Threat Modeling

在这一部分，共有五种不同的场景：

- Access to Target Model
    
    - white box: 记作$M^W$，在这个设定下，敌手能够获得目标模型的所有信息，包括参数和结构。

    - black box: 记作$M^B$，在这个设定下，敌手只能够通过类似API的方式获得输出（一系列工作中也有设定black box是知道模型的架构信息的）。

- Auxiliary Dataset

    - partial training dataset: 记作$D^P_{aux}$，敌手持有部分训练集数据

    - shadow dataset: 记作$D^S_{aux}$，敌手持有和训练集同分布的数据

    - no dataset: 记作$D^N_{aux}$，敌手不持有任何额外数据

- 组合：

    $< M^B, D^S_{aux}>$, $< M^B, D^P_{aux}>$, $< M^W, D^P_{aux}>$, $< M^W, D^S_{aux}>$, $< M^W, D^N_{aux}>$


##### 5. Attacks

1. Membership Inference: MemInf中，敌手的目标是确定一个目标数据是否被用于训练目标模型。即给定一个样本$x_{target}$，可以使用的目标模型M以及一个辅助数据集$D_aux$，MemInf有定义如下：
    
    MemInf : $x_{target}, M, D_{aux} \rightarrow \{member, non-member\}$, where $M \in \{ M^B, M^W \}, D_{aux} \in \{D^P_{aux}, D^S_{aux}\}$

    - Black-Box/Shadow: 敌手首先将shadow dataset分成两个部分，并用其中一个数据集训练一个shadow model（自己设定的），shadow model会给出一个分类的后验概率以及预测的类别。然后将所有的shadow dataset中的所有数据样本送入shadow model做推理并根据输出构建attack model的数据集，如果该样本是训练集中的样本，则标注为member，否则为non-member，此时特征是预测后经过softmax的后验概率，标签即为是否是member。接着将shadow dataset的feature喂入target model里面，获得后验概率和预测结果，将后验概率送入attack model中，以此判断该条数据是否在数据集内。

    ![avatar](../Figure/MemInf.jpeg)
    
    - Black-Box/Partial: 因为有部分真实数据集了，就不需要shadow model来模拟target model了，直接送入attack model进行训练就行了，partial dataset里面的标签全为member，可以增加synthetic数据集，标记为non-member。

    - White-Box/Shadow: 同样要先训练一个shadow model来模拟target model，但是由于他们能够获得target model的所有信息，包括参数和结构，所以在attack中可以加入的参数就更多一些，不仅仅是后验概率，每一层的权重，梯度变化情况也可以作为后续attack model的输入。

    - White-Box/Partial:  因为有部分真实数据集了，就不需要shadow model来模拟target model了，直接送入attack model进行训练就行了，partial dataset里面的标签全为member，可以增加synthetic数据集，标记为non-member。 同时可以利用target model的所有信息，包括参数和结构，所以在attack中可以加入的参数就更多一些，不仅仅是后验概率，每一层的权重，梯度变化情况也可以作为后续attack model的输入。

2. Model Inversion: ModInv旨在重构整个训练数据集，即如果这个方法有效，那么敌手就可以直接对原始数据进行学习。

    ModInv : $ModInv : M^W, D_{aux} \rightarrow \{training samples\}$ where $D_{aux} \in \{D^N_{aux}, D^S_{aux}\}$

    - White-Box/No Auxiliary: 对target model种的每一类，敌手首先创建一个噪声样本，将这些样本喂入模型，然后获得后验概率。然后因为敌手能够知道整个模型训练每一步的反向传播的梯度参数，因此能够根据这些参数反复调整输入样本，直到相关的后验概率满足预设定的阈值，一旦满足了阈值，那么这个优化后的样本将会作为这一类的一个输入样本。

    - White-Box/Shadow: 首先敌手利用一个shadow dataset训练一个GAN模型，接下来敌手优化GAN的生成器中的输入（噪音数据），来找到能够在target model上获得高后验概率的样本，这些样本就是GAN最后的输出。

3. Attribute Inference: 一个机器学习模型可以从原始数据中学习到与原任务不相关的额外(隐私)信息。（例如利用个人资料的照片预测年龄的模型也能够被用于预测人种）

    AttrInf: $x_{target}, M^W, D_{aux} \rightarrow \{target attributes\}$, where D_{aux} \in \{D^P_{aux}, D^S_{aux}\}$

    - White-Box/Shadow and Partial: 这两种方法都是用的是类似的攻击策略，区别在于训练攻击模型过的数据集是什么。在两种情况下，敌手都假定知道输入的属性是什么，然后利用嵌入向量和目标模型里的属性训练一个分类器。

4. Model Stealing: 在模型是黑盒的情况下，根据target model获得参数以达到和target model基本一致的表现。

    ModSteal : $ModSteal : M^B, D_{aux} \rightarrow M^C$ where $M^C is the stolen model and D_{aux} \in \{D^P_{aux}, D^S_{aux}\}$

    - Black-Box/Partial and Shadow: 文章中的Model Stealing主要是对2016年Stealing Machine Learn-
    ing Models via Prediction APIs进行分析。假定知道target model的架构，两种方式使用的攻击策略是相似的，首先利用辅助数据集询问target model获得预测的后验概率，然后以后验概率为目标进行模型训练。

    
##### 6. Experimental Setting:

1. target model: 数据集和模型架构上，mini-batch size为64，交叉熵为损失函数，SGD作为优化器，mometum设置为0.9，weight decay设置为5e-4，每个模型训练300个epoch。前50个epochs的学习旅是1e-2，50-100个epochs的学习率是1e-4直到最后。

2. Membership Inference:

    - Black-Box: attack model有两个输入，一个是排序后的后验概率，一个是二元的指示器标示是否预测正确。每个输入首先送入到不同的2层感知机中，接着获得的两个嵌入向量被拼接到一起然后再送入到一个4层的感知机中。

    - White-Box: attak model有四个输入
        - 排序后的后验概率

        - classification loss

        - target model最后一层的梯度

        - 真实label的独热编码

    > Tips 和黑盒一样，每一个输入送入到不同的神经网络中，拼接起来后送到一个4层的感知机，激活函数为ReLU，batch size设置为64，损失函数为交叉熵损失函数，Adam作为优化器，学习率为1e-5。attack model训练50个epochs，并用准确率，F1 score和AUC作为评估指标。

3. Model Inversion:

    - 对于没有辅助数据集的情况：后验概率的阈值设定为0.999，学习率为1e-2，最大迭代数为3000，early stop策略为100.该攻击只能为每一个类别生成一个样本来表征某一个类别。为了评估这一个样本的质量，首先需要计算每一个类里面的平均样本，然后计算两者之间的均方根误差(MSE)，最后利用所有类别的均方根误差的均值来作为评估指标。*当然这个评估指标会受到标准化策略的影响，例如在FMNIST上的MSE会最大，这是因为里面大部分的值都会标准化为1或者-1*

    - 对于有辅助样本的情况，首先用shadow training dataset训练一个[DCGAN](https://arxiv.org/abs/1511.06434)模型，将生成器的噪声设置为100。对攻击模型，学习率设置为1e-3，momentum设置为0.9，loss ratio为$\lambda$到100，迭代数为1500。评估该攻击模型，需要训练一个评估分类器，并评估reconstructed的样本是否能够被识别出来，评估指标为accuracy和macro-F1 score。

4. Attribute Inference:
 
    - 只评估了两个数据集CelebA和UTKFace，对第一个数据，新任务为预测性别和判别是否年轻。对于UTKFace则选择性别作为新任务。

    - 攻击模型为一个两层的感知机，输入是样本的目标模型最后两层的嵌入向量，优化器为Adam，学习率为1e-3，攻击模型训练50个迭代，评估指标为accuracy和F1 score。

5. Model stealing:

    - 针对20中target model进行实验。
    
    - 每个stolen model使用与target model相同的架构，使用MSE作为损失函数，SGD作为优化器，momentum设置为0.9，学习率为1e-2，共训练50个迭代。准确率以及与目标模型预测结果相同的个数作为评估指标。

![avatar](../Figure/result1.jpg)
![avatar](../Figure/result2.jpg)
![avatar](../Figure/result3.jpg)

##### 7. Defense

DP-SGD（加高斯噪声）和知识蒸馏
![avatar](../Figure/result4.jpg)

DP-SGD和知识蒸馏带来的精度损失

![avatar](../Figure/result5.jpg)